In [ ]:
#- import required python libraries
import netCDF4
import sys, os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import glofrim.glofrim_lib as glib
from os.path import join, isfile, abspath, dirname, basename, normpath
from configparser import ConfigParser
from glofrim.glofrim_lib import configread
import pandas as pd

In [ ]:
% matplotlib notebook
import matplotlib.tri as tri
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib
from matplotlib import cm

def make_map(nrows=1, ncols=1, figsize=(16, 6), projection=ccrs.PlateCarree()):
    subplot_kw = dict(projection=projection)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, subplot_kw=subplot_kw, sharex=True, sharey=True)
    try: # always return list
        iter(axes)
    except TypeError:
        axes = [axes]
    for ax in axes:
        gl = ax.gridlines(draw_labels=True)
        gl.xlabels_top = gl.ylabels_right = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
    return fig, axes

def plot_gtiff(ax, data, transform, cmap='viridis', vmin=None, vmax=None, cbar=False, cbar_kwargs={}, **kwargs):
    extent = plot_extent(data, transform)
    im = ax.imshow(data, extent=extent, vmin=vmin, vmax=vmax, cmap=cmap, origin='upper', **kwargs)
    ax.set_extent(extent)
    
    dmin, dmax = np.nanmin(data), np.nanmax(data)
    vmin = vmin if vmin is not None else dmin 
    vmax = vmax if vmax is not None else dmax
    if (dmax > vmax) and (dmin < vmin):
        cbar_kwargs.update(extend='both')
    elif dmax > vmax:
        cbar_kwargs.update(extend='max')
    elif dmin < vmin:
        cbar_kwargs.update(extend='min')

    if cbar:
        cbar = set_colorbar(ax, im, **cbar_kwargs)
    else:
        cbar = None
    return im, cbar

def plot_extent(data, transform):
    rows, cols = data.shape[-2:]
    left, top = transform * (0, 0)
    right, bottom = transform * (cols, rows)
    extent = (left, right, bottom, top)
    return extent

In [ ]:
from glofrim import Glofrim, PCR, CMF
cbmi = Glofrim()

In [ ]:
#- provide file with local paths to model objects
env_fn = r'../../environment.env'
config = configread(env_fn)
root_dir = config.get('models', 'root_dir')

In [ ]:
#- convert model configs to dictionary
_cf = ConfigParser(inline_comment_prefixes=('#'))
env = glib.configread(abspath(env_fn), encoding='utf-8', cf=_cf)
glib.config2dict(env)

In [ ]:
#- provide file with coupling settings
config_fn = join(root_dir, 'glofrim_PCR2CMF.ini')
# config_fn = join(root_dir, 'glofrim_PCR2DFM.ini')
# config_fn = join(root_dir, 'glofrim_WFL2DFM.ini')
# config_fn = join(root_dir, 'glofrim_PCR2LFP.ini')
config_fn

In [ ]:
#- provide locations where to extract simulated discharge
x_obs_coord = 10.89 # set to coords of Neu-Darchau
y_obs_coord = 53.23
discharge_keys = {'PCR': 'PCR.discharge', 'CMF': 'CMF.outflw', 'DFM': 'DFM.q1', 'LFP': 'LFP.Qx'}

In [ ]:
#- create coupled GLOFRIM model and initialize configs
try:
    cbmi.initialize_config(config_fn, env_fn=env_fn)
except Exception as e:
    print e
    %tb
    sys.exit('configs could not be initialized')

In [ ]:
models = cbmi.bmimodels.keys()
df = pd.DataFrame(columns=models) # output dataframe
print(models)
cbmi.exchanges

In [ ]:
# - provide start and end date
t_start = datetime(2000,1,1)
t_end = datetime(2000,12,31)
cbmi.set_start_time(t_start)
cbmi.set_end_time(t_end)
try:
    t_start == cbmi.get_start_time()
    t_end == cbmi.get_end_time()
except:
    sys.exit('start or end time differ with set_var and get_var')

In [ ]:
#- create coupled GLOFRIM model and initialize models
try:
    cbmi.initialize_model()
except Exception as e:
    print e
    sys.exit('models could not be initialized')

In [ ]:
# cbmi.bmimodels['CMF']._config_fn 

In [ ]:
#- get corresponding indices for models
obs_ind = dict()
obs_xy = dict()
for mod in models:
    if cbmi.bmimodels[mod].grid._1d is not None: # assume 1d for ugrid
        obs_ind[mod] = cbmi.index(x_obs_coord, y_obs_coord, mod, in1d=True)
        obs_xy[mod] = cbmi.bmimodels[mod].grid._1d.xy(ind=obs_ind[mod]).squeeze()
    else: 
        obs_ind[mod] = cbmi.index(x_obs_coord, y_obs_coord, mod, in1d=False)
        obs_xy[mod] = cbmi.bmimodels[mod].grid.xy(ind=obs_ind[mod]) # not for ugrid
obs_ind, obs_xy

In [ ]:
cbmi.get_current_time(), cbmi.get_end_time()

In [ ]:
# # - update models until end time is reached
# # - also, read out discharge data at specified observation point
# # print 'running simulation from ', str(cbmi.get_start_time()),' to ', str(cbmi.get_end_time())
while cbmi.get_current_time() < cbmi.get_end_time():
    t = cbmi.get_current_time()
    cbmi.update()
    for mod in models:
         if 'LFP' in models:
             print cbmi.bmimodels['LFP'].get_current_time()
         df.loc[t, mod] = cbmi.get_value_at_indices(discharge_keys[mod], obs_ind[mod])[0]

In [ ]:
df.tail()

In [ ]:
df.plot()

In [ ]:
# PCR_runoff = cbmi.get_value('PCR.runoff')
# CMF_roffin = cbmi.get_value("CMF.roffin")
# CMF_runoff = cbmi.get_value("CMF.runoff")

# print(np.nansum(PCR_runoff), np.nansum(CMF_roffin), np.nansum(CMF_runoff)) # 0.0056205126668403375
# #- check whether plots are correct
# plt.figure(figsize=(10,10))
# plt.subplot(131)
# plt.imshow(PCR_runoff, vmin=0, vmax=1e-2)
# plt.plot(obs_rc['PCR'][1], obs_rc['PCR'][0], '.r')
# plt.subplot(132)
# plt.imshow(CMF_roffin, vmin=0, vmax=1e-2) # this should look the same at PCR_runoff
# plt.subplot(133)
# # what is the uint of CMF_runoff ??
# plt.imshow(CMF_runoff, vmin=0, vmax=1e2) # this should look similar to PCR_runoff (after interpolation)
# plt.plot(obs_rc['CMF'][1], obs_rc['CMF'][0], '.r')

In [ ]:
#- check dimensions of output maps
try: 
    fig.close()
except: 
    pass
fig, axes = make_map(ncols=len(models), figsize=(10, 3.2))
for i, mod in enumerate(models):
    vals = cbmi.get_value(discharge_keys[mod])
    print(mod, vals.shape)
    if cbmi.bmimodels[mod].grid.type == 2: # assume 1d for ugrid
        coords_1d = cbmi.bmimodels[mod].grid._1d.nodes
        ind_1d = cbmi.bmimodels[mod].grid._1d.inds
        axes[i].scatter(obs_xy[mod][0], obs_xy[mod][1], c='r', s=50)
        axes[i].scatter(coords_1d[:,0], coords_1d[:,1], c=vals[ind_1d], s=0.1, vmin=0, vmax=10)
    else:
        plot_gtiff(axes[i], vals, cbmi.bmimodels[mod].grid.transform, vmin=0, vmax=1e3)
        axes[i].scatter(obs_xy[mod][0], obs_xy[mod][1], c='r', s=50)

In [ ]:
if 'LFP' in models:
    DEMdata = np.ma.masked_greater(cbmi.get_value('LFP.DEM'), 999)
    SGCdata = np.ma.masked_equal(cbmi.get_value('LFP.SGCwidth'), 0)
    Qxdata = np.ma.masked_equal(cbmi.get_value('LFP.H'), 0)
    plt.figure(figsize=(10, 3.2))
    plt.subplot(131)
    plt.title('LFP DEM [m]')
    plt.imshow(DEMdata)
    plt.subplot(132)
    plt.title('LFP SGCwidth [m]')
    plt.imshow(SGCdata)
    plt.subplot(133)
    plt.title('LFP H [m3/s]')
    plt.imshow(Qxdata)

In [ ]:
if 'LFP' in models:
    fig, axes = make_map(ncols=1, figsize=(10, 3.2))
    mod = 'LFP'
    inds = cbmi.bmimodels[mod].grid._1d.inds
    cbmi.bmimodels[mod].set_value_at_indices('DEM', inds, np.ones(inds.shape)*np.nan)
    vals = cbmi.bmimodels[mod].get_value('DEM')
    print(mod, vals.shape)
    plot_gtiff(axes[0], vals, cbmi.bmimodels[mod].grid.transform, vmin=0, vmax=1e3)

In [ ]:
#- finalize all models
try:
    cbmi.finalize()
except Exception as e:
    print e
    sys.exit('models could not be finalized')